# Super Resolution implemetation of Deep Image Prior using Keras

Connecting to Gdrive

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

# File locations
dir = '/content/drive/My Drive/Colab Notebooks/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importing packages

In [0]:
import tensorflow as tf

import numpy as np

from tensorflow.keras.applications import inception_v3
from tensorflow.keras import backend as K
import tensorflow.keras
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Lambda

from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import ZeroPadding2D

from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

K.set_image_data_format('channels_last')
K.image_data_format()

tf.enable_eager_execution()

Loading image

In [0]:
SCALING_FACTOR = 2
encoding_size = 128
perturbation_max = 40


base_dir = '/content/drive/My Drive/Colab Notebooks/deep-image-prior/'
path_to_image = base_dir + 'data/sr/zebra_GT.png'
img_orig = Image.open(path_to_image)
img_orig_np = np.array(img_orig).astype(np.float32) / 255

In [0]:
img_orig_np.shape

(388, 584, 3)

In [0]:
img_bicubic = img_orig.resize((int(img_orig.size[0] * SCALING_FACTOR), int(img_orig.size[1] * SCALING_FACTOR)), Image.BICUBIC)
img_bicubic
img_bicubic_np = np.array(img_bicubic).astype(np.float32) / 255

In [0]:
h, w, ch = img_bicubic_np.shape
if h % 64 != 0:
    crop_h = h // 64 * 64
if w % 64 != 0:
    crop_w = w // 64 * 64

img_bicubic_crop = img_bicubic_np[0:crop_h, 0:crop_w, :]

In [0]:
img_bicubic_crop.shape

(768, 1152, 3)

Define model

In [0]:
input_depth = 32
input_dims = (1, crop_h, crop_w, input_depth)

net_input = K.random_uniform(input_dims) #add var

In [0]:
input_dims

(1, 768, 1152, 32)

In [0]:
# base_model = inception_v3.InceptionV3(weights=None, include_top=False, input_shape=input_dims[1:], pooling='avg')
# x = base_model.get_layer('mixed10').output
# x = ZeroPadding2D((1,1))(x)
# x = Convolution2D(256, 3, padding = 'same', activation = 'relu')(x)
# x = BatchNormalization()(x)
# x = UpSampling2D()(x)
# x = Convolution2D(128, 3, padding = 'same', activation = 'relu')(x)
# x = BatchNormalization()(x)
# x = UpSampling2D()(x)
# x = Convolution2D(64, 3, padding = 'same', activation = 'relu')(x)
# x = BatchNormalization()(x)
# x = UpSampling2D()(x)
# x = Convolution2D(32, 3, padding = 'same', activation = 'relu')(x)
# x = BatchNormalization()(x)
# x = UpSampling2D()(x)
# x = Convolution2D(16, 3, padding = 'same', activation = 'relu')(x)
# x = BatchNormalization()(x)
# x = UpSampling2D()(x)
# x = Convolution2D(3, 3, padding = 'same', activation = 'relu')(x)
# model = Model(inputs=base_model.input, outputs=x)

In [0]:
def UpSampling2DBilinear(size):
    return Lambda(lambda x: tf.image.resize_bilinear(x, size, align_corners=True))

model = Sequential([
    Convolution2D(32, 3, padding = 'same', input_shape = input_dims[1:], activation = 'relu'),
    Convolution2D(32, 3, padding = 'same', activation = 'relu'),
    BatchNormalization(),
    AveragePooling2D(),
    Convolution2D(64, 3, padding = 'same', activation = 'relu'),
    BatchNormalization(),
    AveragePooling2D(),
    Convolution2D(128, 3, padding = 'same', activation = 'relu'),
    BatchNormalization(),
    AveragePooling2D(),
    Convolution2D(256, 3, padding = 'same', activation = 'relu'),
    BatchNormalization(),
    AveragePooling2D(),
    Convolution2D(256, 3, padding = 'same', activation = 'relu'),
    BatchNormalization(),
    AveragePooling2D(),
    Convolution2D(512, 3, padding = 'same', activation = 'relu'),
    BatchNormalization(),
    AveragePooling2D(),
    Convolution2D(512, 3, padding = 'same', activation = 'relu'),
    BatchNormalization(),
    UpSampling2D(),
    Convolution2D(256, 3, padding = 'same', activation = 'relu'),
    BatchNormalization(),
    UpSampling2D(),
    Convolution2D(128, 3, padding = 'same', activation = 'relu'),
    BatchNormalization(),
    UpSampling2D(),
    Convolution2D(64, 3, padding = 'same', activation = 'relu'),
    BatchNormalization(),
    UpSampling2D(),    
    Convolution2D(32, 3, padding = 'same', activation = 'relu'),
    BatchNormalization(),
    UpSampling2D(),
    Convolution2D(16, 3, padding = 'same', activation = 'relu'),
    BatchNormalization(),
    UpSampling2D(),
    Convolution2D(16, 3, padding = 'same', activation = 'relu'),
    BatchNormalization(),
    Convolution2D(3, 3, padding = 'same', activation = 'tanh')
])

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 768, 1152, 32)     9248      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 768, 1152, 32)     9248      
_________________________________________________________________
batch_normalization_v1 (Batc (None, 768, 1152, 32)     128       
_________________________________________________________________
average_pooling2d (AveragePo (None, 384, 576, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 384, 576, 64)      18496     
_________________________________________________________________
batch_normalization_v1_1 (Ba (None, 384, 576, 64)      256       
_________________________________________________________________
average_pooling2d_1 (Average (None, 192, 288, 64)      0         
__________

In [0]:
model.compile(tf.train.AdamOptimizer(), loss = 'mse')

Instructions for updating:
Use tf.cast instead.


In [0]:
n_epochs = 100
img_batch = np.expand_dims(img_bicubic_crop,0)

fit_params = {
    'x': net_input,
    'y': img_batch,
    'epochs': n_epochs,
    'batch_size': 1,
    'verbose': 0,
    'steps_per_epoch': 1
}

In [0]:
img_batch.shape

(1, 768, 1152, 3)

Run model

In [0]:
#TODO add variance to net_input and change randomisation method
#2D bilinear upscaling

In [17]:
for i in range(50):
    model.fit(**fit_params)
    img_pred = model.predict(net_input, steps=1)
    print('Epoch:' + str((i+1) * n_epochs))
    plt.imshow(img_pred[0])
    plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [18]:
import cv2

out_name = 'out/keras_zebra.jpg'
# Save the image to disk
cv2.imwrite(base_dir + out_name, cv2.cvtColor(255 * img_pred[0], cv2.COLOR_RGB2BGR))
#cv2.imwrite(base_dir + out_name, img_pred[0] * 255)

True

In [19]:
out_name = 'out/keras_zebra_bc.jpg'
# Save the image to disk
cv2.imwrite(base_dir + out_name, cv2.cvtColor(255 * img_bicubic_np, cv2.COLOR_RGB2BGR))


True